## Drinking Detection: Masked Images
This Notebook walks through the training, validating, and testing of the CNN framework used for the masked videos

In [ ]:
import tensorflow as tf
import numpy as np
from src import ThreeLayerConvNet, Import_Data, optimizer_init_fn

`Import_Data()` is class used to convert images into tensors. get_data returns X and y tensors for training, testing and validation. get_labels returns the image names for the given set

In [ ]:
importer = Import_Data(train_set = 'masked')
X_train, y_train, X_test, y_test, X_val, y_val = importer.get_data()

ThreeLayerConvNet contains the CNN framework used for training. optimizer_init_fn returns a nesterov SGD keras optimizer

In [ ]:
model = ThreeLayerConvNet()

This block compiles and trains the network

In [ ]:
model.compile(optimizer_init_fn(5e-4),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=16, epochs= 5, validation_data=(X_test, y_test))
#Note we're valdidating on the test set per the task specification
#If satisfactory accuracy isn't reached, increase number of epochs

And this block evaluates how the model does on the validation set and shows the predicted values

In [1]:
loss, acc = model.evaluate(X_val, y_val)
y_val_pred = np.argmax(model.predict(X_val), axis = 1)
print (y_val_pred)

NameError: name 'model' is not defined

Finally, this block writes the training data labels to `output_masked.csv`

In [ ]:
import pandas as pd 

def write_output(path,names, y_pred):
    '''
    names must be a list of length N
    y_pred must be a 1D np.array of size N
    '''
    N = y_pred.shape[0]
    pred_word = []
    for i in range(N): 
        if y_pred[i] == 1: 
            pred_word.append('yes')
        else: 
            pred_word.append('no')
    output_df = pd.DataFrame(columns = ['Name','Drinking'])
    output_df['Name'] = names
    output_df['Drinking'] = y_pred
    output_df.to_csv(path, sep = ',', header = False, index = False)
    print ('Output written to %s' % path)
    
y_train_pred = np.argmax(model.predict(X_train), axis = 1)
write_output('output_masked.csv',importer.get_labels(), y_train_pred)